<a href="https://colab.research.google.com/github/sarahasan17/NLP-Techniques-in-Spam-Classification/blob/main/spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [2]:
import tensorflow

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.keras.utils as ku
from wordcloud import WordCloud
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers

In [4]:
data = pd.read_csv('spam.csv', encoding='latin-1')


In [5]:
import string
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

def remove_punctuation(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct
data['title_wo_punct']=data['v2'].apply(lambda x: remove_punctuation(x))
data.head()

In [6]:
def remove_punctuation(text):
    no_punct=[words for words in text if words not in string.punctuation]
    words_wo_punct=''.join(no_punct)
    return words_wo_punct
data['title_wo_punct']=data['v2'].apply(lambda x: remove_punctuation(x))
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,title_wo_punct
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,Go until jurong point crazy Available only in ...
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,Ok lar Joking wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,U dun say so early hor U c already then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,Nah I dont think he goes to usf he lives aroun...


In [7]:
import re

In [8]:
def tokenize(text):
    split=re.split("\W+",text)
    return split
data['title_wo_punct_split']=data['title_wo_punct'].apply(lambda x: tokenize(x.lower()))
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,title_wo_punct,title_wo_punct_split
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o..."
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t..."
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l..."


In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
stopword = nltk.corpus.stopwords.words('english')

In [11]:
def remove_stopwords(text):
    text=[word for word in text if word not in stopword]
    return text
data['title_wo_punct_split_wo_stopwords'] = data['title_wo_punct'].apply(lambda x: remove_stopwords(x))
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,title_wo_punct,title_wo_punct_split,title_wo_punct_split_wo_stopwords
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o...","[G, , u, n, l, , j, u, r, n, g, , p, n, , ..."
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]","[O, k, , l, r, , J, k, n, g, , w, f, , u, ..."
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[F, r, e, e, , e, n, r, , n, , 2, , , w, ..."
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[U, , u, n, , , , e, r, l, , h, r, , U, ..."
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[N, h, , I, , n, , h, n, k, , h, e, , g, ..."


In [12]:
ps=nltk.PorterStemmer()
def stemming(tokenized_text):
  text=[ps.stem(word) for word in tokenized_text]
  return text
data['title_wo_punct_split_wo_stemming']=data['title_wo_punct_split'].apply(lambda x: stemming(x))
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,title_wo_punct,title_wo_punct_split,title_wo_punct_split_wo_stopwords,title_wo_punct_split_wo_stemming
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o...","[G, , u, n, l, , j, u, r, n, g, , p, n, , ...","[go, until, jurong, point, crazi, avail, onli,..."
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]","[O, k, , l, r, , J, k, n, g, , w, f, , u, ...","[ok, lar, joke, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[F, r, e, e, , e, n, r, , n, , 2, , , w, ...","[free, entri, in, 2, a, wkli, comp, to, win, f..."
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[U, , u, n, , , , e, r, l, , h, r, , U, ...","[u, dun, say, so, earli, hor, u, c, alreadi, t..."
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[N, h, , I, , n, , h, n, k, , h, e, , g, ...","[nah, i, dont, think, he, goe, to, usf, he, li..."


In [13]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [14]:
wn=nltk.WordNetLemmatizer()
def lemmatizing(tokenized_text):
  text=[wn.lemmatize(word) for word in tokenized_text]
  return text
data['text_wo_lemmatizer']=data['title_wo_punct_split_wo_stemming'].apply(lambda x: lemmatizing(x))
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,title_wo_punct,title_wo_punct_split,title_wo_punct_split_wo_stopwords,title_wo_punct_split_wo_stemming,text_wo_lemmatizer
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o...","[G, , u, n, l, , j, u, r, n, g, , p, n, , ...","[go, until, jurong, point, crazi, avail, onli,...","[go, until, jurong, point, crazi, avail, onli,..."
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]","[O, k, , l, r, , J, k, n, g, , w, f, , u, ...","[ok, lar, joke, wif, u, oni]","[ok, lar, joke, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[F, r, e, e, , e, n, r, , n, , 2, , , w, ...","[free, entri, in, 2, a, wkli, comp, to, win, f...","[free, entri, in, 2, a, wkli, comp, to, win, f..."
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[U, , u, n, , , , e, r, l, , h, r, , U, ...","[u, dun, say, so, earli, hor, u, c, alreadi, t...","[u, dun, say, so, earli, hor, u, c, alreadi, t..."
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[N, h, , I, , n, , h, n, k, , h, e, , g, ...","[nah, i, dont, think, he, goe, to, usf, he, li...","[nah, i, dont, think, he, goe, to, usf, he, li..."


In [15]:
from nltk.corpus import stopwords

In [16]:
spam_messages = data[data["v1"] == "spam"]["text_wo_lemmatizer"]
ham_messages = data[data["v1"] == "ham"]["text_wo_lemmatizer"]

In [17]:
print("SPAM MESSAGES:",len(spam_messages))
print("HAM MESSAGES:",len(ham_messages))

SPAM MESSAGES: 747
HAM MESSAGES: 4825


In [18]:
print(spam_messages)

2       [free, entri, in, 2, a, wkli, comp, to, win, f...
5       [freemsg, hey, there, darl, it, been, 3, week,...
8       [winner, a, a, valu, network, custom, you, hav...
9       [had, your, mobil, 11, month, or, more, u, r, ...
11      [six, chanc, to, win, cash, from, 100, to, 200...
                              ...                        
5537    [want, explicit, sex, in, 30, sec, ring, 02073...
5540    [ask, 3mobil, if, 0870, chatlin, inclu, in, fr...
5547    [had, your, contract, mobil, 11, mnth, latest,...
5566    [remind, from, o2, to, get, 250, pound, free, ...
5567    [thi, is, the, 2nd, time, we, have, tri, 2, co...
Name: text_wo_lemmatizer, Length: 747, dtype: object


In [19]:
#spam_data=[word for word in data if data['v1']=='spam']
spam_data = data.loc[data['v1'] == 'spam']

In [20]:
spam_data[:10]

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,title_wo_punct,title_wo_punct_split,title_wo_punct_split_wo_stopwords,title_wo_punct_split_wo_stemming,text_wo_lemmatizer
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[F, r, e, e, , e, n, r, , n, , 2, , , w, ...","[free, entri, in, 2, a, wkli, comp, to, win, f...","[free, entri, in, 2, a, wkli, comp, to, win, f..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...,NaN,NaN,NaN,FreeMsg Hey there darling its been 3 weeks now...,"[freemsg, hey, there, darling, its, been, 3, w...","[F, r, e, e, M, g, , H, e, , h, e, r, e, , ...","[freemsg, hey, there, darl, it, been, 3, week,...","[freemsg, hey, there, darl, it, been, 3, week,..."
8,spam,WINNER!! As a valued network customer you have...,NaN,NaN,NaN,WINNER As a valued network customer you have b...,"[winner, as, a, valued, network, customer, you...","[W, I, N, N, E, R, , A, , , v, l, u, e, , ...","[winner, as, a, valu, network, custom, you, ha...","[winner, a, a, valu, network, custom, you, hav..."
9,spam,Had your mobile 11 months or more? U R entitle...,NaN,NaN,NaN,Had your mobile 11 months or more U R entitled...,"[had, your, mobile, 11, months, or, more, u, r...","[H, , u, r, , b, l, e, , 1, 1, , n, h, , ...","[had, your, mobil, 11, month, or, more, u, r, ...","[had, your, mobil, 11, month, or, more, u, r, ..."
11,spam,"SIX chances to win CASH! From 100 to 20,000 po...",NaN,NaN,NaN,SIX chances to win CASH From 100 to 20000 poun...,"[six, chances, to, win, cash, from, 100, to, 2...","[S, I, X, , c, h, n, c, e, , , w, n, , C, ...","[six, chanc, to, win, cash, from, 100, to, 200...","[six, chanc, to, win, cash, from, 100, to, 200..."
12,spam,URGENT! You have won a 1 week FREE membership ...,NaN,NaN,NaN,URGENT You have won a 1 week FREE membership i...,"[urgent, you, have, won, a, 1, week, free, mem...","[U, R, G, E, N, T, , Y, u, , h, v, e, , w, ...","[urgent, you, have, won, a, 1, week, free, mem...","[urgent, you, have, won, a, 1, week, free, mem..."
15,spam,"XXXMobileMovieClub: To use your credit, click ...",NaN,NaN,NaN,XXXMobileMovieClub To use your credit click th...,"[xxxmobilemovieclub, to, use, your, credit, cl...","[X, X, X, M, b, l, e, M, v, e, C, l, u, b, , ...","[xxxmobilemovieclub, to, use, your, credit, cl...","[xxxmobilemovieclub, to, use, your, credit, cl..."
19,spam,England v Macedonia - dont miss the goals/team...,NaN,NaN,NaN,England v Macedonia dont miss the goalsteam n...,"[england, v, macedonia, dont, miss, the, goals...","[E, n, g, l, n, , v, , M, c, e, n, , , n, ...","[england, v, macedonia, dont, miss, the, goals...","[england, v, macedonia, dont, miss, the, goals..."
34,spam,Thanks for your subscription to Ringtone UK yo...,NaN,NaN,NaN,Thanks for your subscription to Ringtone UK yo...,"[thanks, for, your, subscription, to, ringtone...","[T, h, n, k, , f, r, , u, r, , u, b, c, r, ...","[thank, for, your, subscript, to, rington, uk,...","[thank, for, your, subscript, to, rington, uk,..."
42,spam,07732584351 - Rodger Burns - MSG = We tried to...,NaN,NaN,NaN,07732584351 Rodger Burns MSG We tried to ca...,"[07732584351, rodger, burns, msg, we, tried, t...","[0, 7, 7, 3, 2, 5, 8, 4, 3, 5, 1, , , R, g, ...","[07732584351, rodger, burn, msg, we, tri, to, ...","[07732584351, rodger, burn, msg, we, tri, to, ..."


In [21]:
spam_data_list=[]
for i in  spam_data['text_wo_lemmatizer']:
  for j in i:
    if j.lower() not in stopwords.words('english') and j.isalpha():
      spam_data_list.append(j)

In [22]:
spam_data_list[:10]

['free', 'entri', 'wkli', 'comp', 'win', 'fa', 'cup', 'final', 'tkt', 'may']

In [23]:
print("Most used spam words are:\n")
print(pd.Series(spam_data_list).value_counts().head(10))

Most used spam words are:

call     366
å        291
free     216
txt      163
u        156
ur       144
text     138
mobil    135
claim    115
stop     113
dtype: int64


In [24]:
ham_data = data.loc[data['v1'] == 'ham']

In [25]:
ham_data[:10]

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,title_wo_punct,title_wo_punct_split,title_wo_punct_split_wo_stopwords,title_wo_punct_split_wo_stemming,text_wo_lemmatizer
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o...","[G, , u, n, l, , j, u, r, n, g, , p, n, , ...","[go, until, jurong, point, crazi, avail, onli,...","[go, until, jurong, point, crazi, avail, onli,..."
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]","[O, k, , l, r, , J, k, n, g, , w, f, , u, ...","[ok, lar, joke, wif, u, oni]","[ok, lar, joke, wif, u, oni]"
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[U, , u, n, , , , e, r, l, , h, r, , U, ...","[u, dun, say, so, earli, hor, u, c, alreadi, t...","[u, dun, say, so, earli, hor, u, c, alreadi, t..."
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[N, h, , I, , n, , h, n, k, , h, e, , g, ...","[nah, i, dont, think, he, goe, to, usf, he, li...","[nah, i, dont, think, he, goe, to, usf, he, li..."
6,ham,Even my brother is not like to speak with me. ...,NaN,NaN,NaN,Even my brother is not like to speak with me T...,"[even, my, brother, is, not, like, to, speak, ...","[E, v, e, n, , , b, r, h, e, r, , , n, , ...","[even, my, brother, is, not, like, to, speak, ...","[even, my, brother, is, not, like, to, speak, ..."
7,ham,As per your request 'Melle Melle (Oru Minnamin...,NaN,NaN,NaN,As per your request Melle Melle Oru Minnaminun...,"[as, per, your, request, melle, melle, oru, mi...","[A, , p, e, r, , u, r, , r, e, q, u, e, , ...","[as, per, your, request, mell, mell, oru, minn...","[a, per, your, request, mell, mell, oru, minna..."
10,ham,I'm gonna be home soon and i don't want to tal...,NaN,NaN,NaN,Im gonna be home soon and i dont want to talk ...,"[im, gonna, be, home, soon, and, i, dont, want...","[I, , g, n, n, , b, e, , h, e, , n, , n, ...","[im, gonna, be, home, soon, and, i, dont, want...","[im, gonna, be, home, soon, and, i, dont, want..."
13,ham,I've been searching for the right words to tha...,NaN,NaN,NaN,Ive been searching for the right words to than...,"[ive, been, searching, for, the, right, words,...","[I, v, e, , b, e, e, n, , e, r, c, h, n, g, ...","[ive, been, search, for, the, right, word, to,...","[ive, been, search, for, the, right, word, to,..."
14,ham,I HAVE A DATE ON SUNDAY WITH WILL!!,NaN,NaN,NaN,I HAVE A DATE ON SUNDAY WITH WILL,"[i, have, a, date, on, sunday, with, will]","[I, , H, A, V, E, , A, , D, A, T, E, , O, ...","[i, have, a, date, on, sunday, with, will]","[i, have, a, date, on, sunday, with, will]"
16,ham,Oh k...i'm watching here:),NaN,NaN,NaN,Oh kim watching here,"[oh, kim, watching, here]","[O, h, , k, , w, c, h, n, g, , h, e, r, e]","[oh, kim, watch, here]","[oh, kim, watch, here]"


In [26]:
ham_data_list=[]
for i in  ham_data['text_wo_lemmatizer']:
  for j in i:
    if j.lower() not in stopwords.words('english') and j.isalpha():
      ham_data_list.append(j)

In [27]:
ham_data_list[:10]

['go',
 'jurong',
 'point',
 'crazi',
 'avail',
 'onli',
 'bugi',
 'n',
 'great',
 'world']

In [28]:
print("Most used ham words are:\n")
print(pd.Series(ham_data_list).value_counts().head(10))

Most used ham words are:

u       1027
im       450
go       414
get      360
come     292
call     287
ltgt     276
ok       272
dont     257
thi      245
dtype: int64


**VISUALIZATION**

In [29]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,title_wo_punct,title_wo_punct_split,title_wo_punct_split_wo_stopwords,title_wo_punct_split_wo_stemming,text_wo_lemmatizer
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o...","[G, , u, n, l, , j, u, r, n, g, , p, n, , ...","[go, until, jurong, point, crazi, avail, onli,...","[go, until, jurong, point, crazi, avail, onli,..."
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]","[O, k, , l, r, , J, k, n, g, , w, f, , u, ...","[ok, lar, joke, wif, u, oni]","[ok, lar, joke, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[F, r, e, e, , e, n, r, , n, , 2, , , w, ...","[free, entri, in, 2, a, wkli, comp, to, win, f...","[free, entri, in, 2, a, wkli, comp, to, win, f..."
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[U, , u, n, , , , e, r, l, , h, r, , U, ...","[u, dun, say, so, earli, hor, u, c, alreadi, t...","[u, dun, say, so, earli, hor, u, c, alreadi, t..."
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[N, h, , I, , n, , h, n, k, , h, e, , g, ...","[nah, i, dont, think, he, goe, to, usf, he, li...","[nah, i, dont, think, he, goe, to, usf, he, li..."


In [38]:
c=0
data["new_val"] = len(data)*""
for i in data["text_wo_lemmatizer"]:
  l=""
  for j in i:
    l+=j+" "
  data["new_val"][c]=l.strip()
  c+=1

In [39]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4,title_wo_punct,title_wo_punct_split,title_wo_punct_split_wo_stopwords,title_wo_punct_split_wo_stemming,text_wo_lemmatizer,new_val
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN,Go until jurong point crazy Available only in ...,"[go, until, jurong, point, crazy, available, o...","[G, , u, n, l, , j, u, r, n, g, , p, n, , ...","[go, until, jurong, point, crazi, avail, onli,...","[go, until, jurong, point, crazi, avail, onli,...",go until jurong point crazi avail onli in bugi...
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN,Ok lar Joking wif u oni,"[ok, lar, joking, wif, u, oni]","[O, k, , l, r, , J, k, n, g, , w, f, , u, ...","[ok, lar, joke, wif, u, oni]","[ok, lar, joke, wif, u, oni]",ok lar joke wif u oni
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f...","[F, r, e, e, , e, n, r, , n, , 2, , , w, ...","[free, entri, in, 2, a, wkli, comp, to, win, f...","[free, entri, in, 2, a, wkli, comp, to, win, f...",free entri in 2 a wkli comp to win fa cup fina...
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN,U dun say so early hor U c already then say,"[u, dun, say, so, early, hor, u, c, already, t...","[U, , u, n, , , , e, r, l, , h, r, , U, ...","[u, dun, say, so, earli, hor, u, c, alreadi, t...","[u, dun, say, so, earli, hor, u, c, alreadi, t...",u dun say so earli hor u c alreadi then say
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN,Nah I dont think he goes to usf he lives aroun...,"[nah, i, dont, think, he, goes, to, usf, he, l...","[N, h, , I, , n, , h, n, k, , h, e, , g, ...","[nah, i, dont, think, he, goe, to, usf, he, li...","[nah, i, dont, think, he, goe, to, usf, he, li...",nah i dont think he goe to usf he live around ...


In [40]:
from sklearn.feature_extraction.text import CountVectorizer

In [71]:
# Creating the Bag of Words

# Note the here we are passing already process messages (after removing punctuations and stopwords)
# Creating the Bag of Words
# Note that we are passing already processed messages (after removing punctuation and stopwords)

vectorizer = CountVectorizer()
bow_transformer = vectorizer.fit(data['new_val'])

print("20 Bag of Words (BOW) Features: \n")
print(vectorizer.get_feature_names_out()[920:940])

print("\nTotal number of vocab words: ", len(vectorizer.vocabulary_))


20 Bag of Words (BOW) Features: 

['aah' 'aaniy' 'aaooooright' 'aathilov' 'aathiwher' 'ab' 'abbey' 'abdomen'
 'abeg' 'abel' 'aberdeen' 'abi' 'abil' 'abiola' 'abj' 'abl' 'abnorm'
 'about' 'abouta' 'abov']

Total number of vocab words:  8093


In [45]:
print(data['new_val'].head())

0    go until jurong point crazi avail onli in bugi...
1                                ok lar joke wif u oni
2    free entri in 2 a wkli comp to win fa cup fina...
3          u dun say so earli hor u c alreadi then say
4    nah i dont think he goe to usf he live around ...
Name: new_val, dtype: object


In [ ]:
message4 = messages['message'][3]
print(message4)

In [72]:
bow4 = bow_transformer.transform([data['new_val'][3]])
print(bow4)
print(bow4.shape)

  (0, 1109)	1
  (0, 2655)	1
  (0, 2673)	1
  (0, 3684)	1
  (0, 6163)	2
  (0, 6498)	1
  (0, 7047)	1
(1, 8093)


In [73]:
print(bow_transformer.get_feature_names_out()[918])

a30


In [74]:
messages_bow = bow_transformer.transform(data['new_val'])

In [75]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(messages_bow)

In [76]:
tfidf4 = tfidf_transformer.transform(bow4)
print(tfidf4)

  (0, 7047)	0.24735151263970326
  (0, 6498)	0.21144061991302787
  (0, 6163)	0.5576788481687237
  (0, 3684)	0.5005134980056432
  (0, 2673)	0.3615705296303255
  (0, 2655)	0.33307269508911014
  (0, 1109)	0.30152937826010995


In [77]:
print(tfidf_transformer.idf_[bow_transformer.vocabulary_['say']])

4.750491464368385


In [78]:
messages_tfidf = tfidf_transformer.transform(messages_bow)
print(messages_tfidf.shape)

(5572, 8093)


In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer

vec = TfidfVectorizer(encoding = "latin-1", strip_accents = "unicode", stop_words = "english")
features = vec.fit_transform(data["new_val"])
print(features.shape)

print(len(vec.vocabulary_))

(5572, 7890)
7890


In [80]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [82]:
msg_train, msg_test, label_train, label_test = \
train_test_split(messages_tfidf, data['v1'], test_size=0.2)

In [83]:


print("train dataset features size : ",msg_train.shape)
print("train dataset label size", label_train.shape)

print("\n")

print("test dataset features size", msg_test.shape)
print("test dataset lable size", label_test.shape)



train dataset features size :  (4457, 8093)
train dataset label size (4457,)


test dataset features size (1115, 8093)
test dataset lable size (1115,)


In [84]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
spam_detect_model = clf.fit(msg_train, label_train)



In [85]:
predict_train = spam_detect_model.predict(msg_train)

In [86]:
print("Classification Report \n",metrics.classification_report(label_train, predict_train))
print("\n")
print("Confusion Matrix \n",metrics.confusion_matrix(label_train, predict_train))
print("\n")
print("Accuracy of Train dataset : {0:0.3f}".format(metrics.accuracy_score(label_train, predict_train)))

Classification Report 
               precision    recall  f1-score   support

         ham       0.96      1.00      0.98      3862
        spam       1.00      0.73      0.84       595

    accuracy                           0.96      4457
   macro avg       0.98      0.86      0.91      4457
weighted avg       0.96      0.96      0.96      4457



Confusion Matrix 
 [[3862    0]
 [ 163  432]]


Accuracy of Train dataset : 0.963


In [89]:
print('predicted:', spam_detect_model.predict(tfidf4)[0])
print('expected:', data['v1'][3])

predicted: ham
expected: ham


In [90]:
label_predictions = spam_detect_model.predict(msg_test)
print(label_predictions)

['ham' 'ham' 'ham' ... 'ham' 'ham' 'ham']


In [91]:
print(metrics.classification_report(label_test, label_predictions))
print(metrics.confusion_matrix(label_test, label_predictions))

              precision    recall  f1-score   support

         ham       0.95      1.00      0.97       963
        spam       1.00      0.65      0.79       152

    accuracy                           0.95      1115
   macro avg       0.97      0.83      0.88      1115
weighted avg       0.95      0.95      0.95      1115

[[963   0]
 [ 53  99]]


In [92]:
# Printing the Overall Accuracy of the model
print("Accuracy of the model : {0:0.3f}".format(metrics.accuracy_score(label_test, label_predictions)))

Accuracy of the model : 0.952
